In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9377,2024-04-16,Porto Rico Bsn,21:00,Osos de Manati,Criollos de Caguas,1.86,1.86,179.5,1.85,1.85,0.0,0.00,0.00,hOOS9BXj,0.537634,0.537634,0.540541,0.540541,0.075269,226.896,70.215988,0.309463,1.2,1.643168,1.369306,264.33,2.352,0.699800,0.297534,-43.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,89,81,2.97,2.15,0.000,0.000,0.000000,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9378,2024-04-16,Porto Rico Bsn,21:00,Mets de Guaynabo,Capitanes De Arecibo,1.81,1.91,175.5,1.85,1.85,0.0,0.00,0.00,6RCAYfAi,0.552486,0.523560,0.540541,0.540541,0.076046,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,94,105,1.37,1.94,0.000,0.000,0.038016,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9379,2024-04-16,Porto Rico Bsn,21:00,Indios de Mayaguez,Cangrejeros,1.51,2.45,175.5,1.85,1.85,0.0,0.00,0.00,pIBEXEPc,0.662252,0.408163,0.540541,0.540541,0.070415,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,90,82,1.41,1.55,0.000,0.000,0.335697,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9380,2024-04-16,Letônia Lbl,13:00,BK Ogre,BK Liepaja,1.31,3.32,160.5,1.87,1.79,0.0,0.00,0.00,MmY6sdxH,0.763359,0.301205,0.534759,0.558659,0.064564,322.814,256.934408,0.795921,1.2,1.643168,1.369306,134.64,3.930,3.218035,0.818838,-12.0,195.930,126.395165,0.645104,1.2,1.643168,1.369306,155.21,2.450,1.595885,0.651382,4.0,72,83,1.87,1.87,405.112,320.568,0.613946,0.030912,NaN,-0.20,-0.040,-7.750000,0.557217,0.6,0.042783,-3.75,-0.750,-3.093333,0.458992,0.2,-0.258992
9381,2024-04-16,México Cibacopa,23:15,Astros,Pioneros de Los Mochis,1.23,3.86,176.5,1.83,1.87,0.0,0.00,0.00,t2rS2DLt,0.813008,0.259067,0.546448,0.534759,0.072075,148.764,20.896375,0.140467,1.8,1.643168,0.912871,139.36,1.568,0.240146,0.153154,46.0,178.648,42.262353,0.236568,1.8,1.643168,0.912871,245.34,1.962,0.432978,0.220682,1.0,104,94,1.34,2.61,121.996,189.256,0.730723,0.015289,NaN,0.01,0.002,115.000000,0.000000,0.0,0.000000,-0.98,-0.196,-14.591837,0.000000,0.0,0.000000
9382,2024-04-16,Lituânia Moteru Lyga Feminina,12:30,LCC F,Kibirkstis Vilnius F,1.30,3.31,140.5,1.85,1.85,0.0,0.00,0.00,0UwtFKtP,0.769231,0.302115,0.540541,0.540541,0.071346,91.708,21.505938,0.234504,3.0,0.000000,0.000000,78.10,1.188,0.310435,0.261309,52.0,939.898,1114.069526,1.185309,2.4,1.341641,0.559017,102.81,11.466,13.282115,1.158391,102.0,71,69,1.10,1.49,0.000,940.604,0.616609,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,50.78,10.156,0.227452,0.000000,0.0,0.000000
9383,2024-04-16,Lituânia Lkl,12:15,Lietkabelis,Pieno Zvaigzdes,1.09,6.36,174.5,1.86,1.84,0.0,0.00,0.00,WtUukgkQ,0.917431,0.157233,0.537634,0.543478,0.074664,195.156,76.356711,0.391260,0.6,1.341641,2.236068,162.96,2.284,0.842603,0.368916,-25.0,392.428,286.522328,0.730127,0.0,0.000000,NaN,292.99,4.984,3.753429,0.753096,-43.0,84,83,1.94,3.53,147.288,435.806,1.000390,0.007644,NaN,-2.54,-0.508,-0.177165,0.000000,0.0,0.000000,-5.00,-1.000,-5.360000,0.000000,0.0,0.000000
9384,2024-04-16,Reino Unido Wbbl (Fem.),1

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9421,14:45,Dinamarca Kvindebasketligaen Feminina,BMS Herlev F,Amager F,1.64,Back Home
